## Carregando dados de um arquivo JSON

 * Vamos analisar dados dentro de um arquivo JSON, dados mensagens sobre análise de sentimentos:
 
 1. Carregar o arquivo de dados JSON
 2. Entender o Schema gerado pelo arquivo
 3. Separá-lo em fragmentos
 4. Criar uma visão com os dados para analisar dentro do SPARK SQL
 5. Mostrar as frases com sentimentos negativos
 
 

In [ ]:
#importando sparksession
from pyspark.sql import SparkSession

#criando um objeto sparksession object e um appName 
sparkSession=SparkSession.builder.appName("sentimento").getOrCreate()

In [ ]:
# Carregando os dados arquivo JSON
path = "/Treinamento/Dados/sentimento.json"
sentimentoDF = sparkSession.read.json(path,multiLine = "true")

In [ ]:
#Exibindo os dados arquivo JSON
sentimentoDF.show()

In [ ]:
#Exibindo o schema arquivo JSON
sentimentoDF.printSchema()

In [ ]:
# Especificando quais campos deseja trabalhar
sent2DF = sentimentoDF.select("context","qas.id", "qas.is_impossible","qas.question")
sent2DF.printSchema()

In [ ]:
# Exibindo os registros
sent2DF.show(10, False)

In [ ]:
# Separando em apenas 2 campos para uso no SQL
sent3DF = sentimentoDF.select("context", "qas.question")
sent3DF.show()

In [ ]:
# Criando uma visão com os dados
sent3DF.createOrReplaceTempView("sentimentos")

In [ ]:
#Pesquisando dentre os dados todos que possuem a palavra HOPE
sparkSession.sql("select context from sentimentos where context like '%hope%'").show()

In [ ]:
# COnvertendo os dados de array para string coma  função concat_ws
sent4DF = sparkSession.sql("select context, question, concat_ws(',',question)" + \
                 " as tps from sentimentos")

In [ ]:
# Criando uma visão com os dados
sent4DF.createOrReplaceTempView("tipo_sentimento")

In [ ]:
#Pesquisando dentre os dados todos que possuem o tipo de sentimento negativo
sparkSession.sql("select context, question from tipo_sentimento where tps = 'negative'").show()

In [ ]:
#Contagem dos tipos de sentimento
sparkSession.sql("select tps, count(*) from tipo_sentimento group by tps").show()